In [1]:
!pip install obspy

In [2]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = "/content/drive/My Drive/R_Theory_4th_Sem_BTech/lunar/training/catalogs/apollo12_catalog_GradeA_final.csv"

data = pd.read_csv(file_path)

In [6]:
data.head()

,filename,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),evid,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,1970-01-19T20:25:00.000000,73500.0,evid00002,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,1970-03-25T03:32:00.000000,12720.0,evid00003,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,1970-03-26T20:17:00.000000,73020.0,evid00004,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,1970-04-25T01:14:00.000000,4440.0,evid00006,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,1970-04-26T14:29:00.000000,52140.0,evid00007,deep_mq


In [7]:
file_name = data['filename']
file_name

,filename
0,xa.s12.00.mhz.1970-01-19HR00_evid00002
1,xa.s12.00.mhz.1970-03-25HR00_evid00003
2,xa.s12.00.mhz.1970-03-26HR00_evid00004
3,xa.s12.00.mhz.1970-04-25HR00_evid00006
4,xa.s12.00.mhz.1970-04-26HR00_evid00007
...,...
71,xa.s12.00.mhz.1974-10-14HR00_evid00156
72,xa.s12.00.mhz.1975-04-12HR00_evid00191
73,xa.s12.00.mhz.1975-05-04HR00_evid00192
74,xa.s12.00.mhz.1975-06-24HR00_evid00196


In [8]:
data_directory = "/content/drive/My Drive/R_Theory_4th_Sem_BTech/lunar/training/data/S12_GradeA"

In [9]:
csv_file = f'{data_directory}/{file_name[0]}.csv'
data_cat = pd.read_csv(csv_file)
data_cat

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s)
0,1970-01-19T00:00:00.665000,0.000000,-6.153279e-14
1,1970-01-19T00:00:00.815943,0.150943,-7.701288e-14
2,1970-01-19T00:00:00.966887,0.301887,-8.396187e-14
3,1970-01-19T00:00:01.117830,0.452830,-8.096155e-14
4,1970-01-19T00:00:01.268774,0.603774,-7.097599e-14
...,...,...,...
572410,1970-01-20T00:00:02.174434,86401.509434,-1.472713e-14
572411,1970-01-20T00:00:02.325377,86401.660377,-1.956104e-14
572412,1970-01-20T00:00:02.476321,86401.811321,-2.240307e-14
572413,1970-01-20T00:00:02.627264,86401.962264,-2.998405e-14


In [10]:
mseed_file = f'{data_directory}/{file_name[0]}.mseed'
read(mseed_file)

1 Trace(s) in Stream:
XA.S12.00.MHZ | 1970-01-19T00:00:00.665000Z - 1970-01-20T00:00:02.778208Z | 6.6 Hz, 572415 samples

In [11]:
from obspy.signal.trigger import classic_sta_lta

# Define paths
base_path = "/content/drive/My Drive/R_Theory_4th_Sem_BTech/lunar/training"
catalog_path = f"{base_path}/catalogs/apollo12_catalog_GradeA_final.csv"
data_directory = f"{base_path}/data/S12_GradeA"

# Load the catalog CSV
data = pd.read_csv(catalog_path)

# Get filenames and corresponding mq_type
file_names = data['filename'].tolist()
mq_types = data['mq_type'].tolist()

# Initialize feature list
features_list = []

# Iterate through each file name
for name in file_names:
    csv_file = f"{data_directory}/{name}.csv"

    try:
        # Read the CSV file
        data_cat = pd.read_csv(csv_file)

        # Extract time and velocity
        csv_times = np.array(data_cat['time_rel(sec)'].tolist())
        csv_data = np.array(data_cat['velocity(m/s)'].tolist())

        # Arrival time from the catalog
        arrival_time = data.iloc[0]['time_rel(sec)']
        time_to_arrival = csv_times - arrival_time

        # Compute features
        mean_velocity = np.mean(csv_data)
        std_velocity = np.std(csv_data)
        max_velocity = np.max(csv_data)
        min_velocity = np.min(csv_data)
        range_velocity = max_velocity - min_velocity
        median_velocity = np.median(csv_data)
        rms_velocity = np.sqrt(np.mean(csv_data ** 2))
        energy = np.sum(csv_data ** 2)

        # Impulse factor
        threshold = np.percentile(csv_data, 95)
        impulse_events = csv_data[csv_data > threshold]
        impulse_factor = len(impulse_events) / len(csv_data)

        # Velocity derivative
        velocity_derivative = np.diff(csv_data) / np.diff(csv_times)

        # Window around arrival time
        window_size = 5  # seconds
        start_index = np.searchsorted(csv_times, arrival_time - window_size)
        end_index = np.searchsorted(csv_times, arrival_time + window_size)

        # Extract window data
        window_data = csv_data[start_index:end_index]
        mean_window_velocity = np.mean(window_data)
        max_window_velocity = np.max(window_data)

        # STA/LTA calculation
        sta_window = 1  # short-term window (in seconds)
        lta_window = 10  # long-term window (in seconds)

        # Convert STA and LTA windows to number of samples based on sampling frequency
        sampling_rate = 1 / np.mean(np.diff(csv_times))
        sta_samples = int(sta_window * sampling_rate)
        lta_samples = int(lta_window * sampling_rate)

        # Compute STA/LTA characteristic function
        cft = classic_sta_lta(csv_data, sta_samples, lta_samples)

        # Maximum STA/LTA value
        max_sta_lta = np.max(cft)

        # Store features in a dictionary
        features = {
            'File Name': name,
            'Mean Velocity': mean_velocity,
            'Standard Deviation': std_velocity,
            'Max Velocity': max_velocity,
            'Min Velocity': min_velocity,
            'Range Velocity': range_velocity,
            'Median Velocity': median_velocity,
            'RMS Velocity': rms_velocity,
            'Energy': energy,
            'Impulse Factor': impulse_factor,
            'Velocity Derivative Mean': np.mean(velocity_derivative) if len(velocity_derivative) > 0 else np.nan,
            'Velocity Derivative Std': np.std(velocity_derivative) if len(velocity_derivative) > 0 else np.nan,
            'Mean Velocity Around Arrival': mean_window_velocity,
            'Max Velocity Around Arrival': max_window_velocity,
            'Arrival Time': arrival_time,
            'Max STA/LTA': max_sta_lta
        }

        # Append features to the list
        features_list.append(features)

    except FileNotFoundError:
        print(f"File not found: {csv_file}. Skipping this file.")
    except Exception as e:
        print(f"An error occurred while processing {csv_file}: {e}")

# Create DataFrame from features list
features_df = pd.DataFrame(features_list)

# Create DataFrame for mq_type
mq_types_df = pd.DataFrame({'File Name': file_names, 'mq_type': mq_types})

# Merge features_df with mq_types_df on 'File Name'
combined_df = pd.merge(features_df, mq_types_df, on='File Name', how='left')

# Save the DataFrame as a CSV file
combined_df


File not found: /content/drive/My Drive/R_Theory_4th_Sem_BTech/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1971-04-13HR00_evid00029.csv. Skipping this file.


/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


An error occurred while processing /content/drive/My Drive/R_Theory_4th_Sem_BTech/lunar/training/data/S12_GradeA/xa.s12.00.mhz.1973-03-13HR00_evid00094.csv: zero-size array to reduction operation maximum which has no identity


,File Name,Mean Velocity,Standard Deviation,Max Velocity,Min Velocity,Range Velocity,Median Velocity,RMS Velocity,Energy,Impulse Factor,Velocity Derivative Mean,Velocity Derivative Std,Mean Velocity Around Arrival,Max Velocity Around Arrival,Arrival Time,Max STA/LTA,mq_type
0,xa.s12.00.mhz.1970-01-19HR00_evid00002,-8.443134e-13,3.530056e-10,7.874026e-09,-8.185283e-09,1.605931e-08,-1.633815e-17,3.530066e-10,7.133071e-14,0.050000,1.953538e-19,1.617267e-09,-2.976370e-11,1.509733e-09,73500.0,10.200389,impact_mq
1,xa.s12.00.mhz.1970-03-25HR00_evid00003,-1.939339e-12,3.865137e-10,4.707866e-09,-4.603228e-09,9.311095e-09,1.764188e-12,3.865185e-10,8.551625e-14,0.050001,2.327948e-20,1.300470e-09,2.869342e-11,8.028672e-10,73500.0,10.244349,impact_mq
2,xa.s12.00.mhz.1970-03-26HR00_evid00004,-2.980386e-13,3.219582e-10,5.969005e-09,-6.144452e-09,1.211346e-08,3.116433e-16,3.219583e-10,5.933450e-14,0.050001,1.108431e-19,1.377912e-09,-7.111081e-11,3.978438e-09,73500.0,10.160171,impact_mq
3,xa.s12.00.mhz.1970-04-25HR00_evid00006,-1.547089e-13,3.383782e-10,6.853803e-09,-6.155705e-09,1.300951e-08,3.515417e-14,3.383782e-10,6.554142e-14,0.050000,-7.894813e-20,1.571516e-09,-3.392656e-12,4.175311e-10,73500.0,10.082999,impact_mq
4,xa.s12.00.mhz.1970-04-26HR00_evid00007,-6.921802e-13,3.009879e-10,5.491012e-09,-4.475551e-09,9.966563e-09,-3.369827e-13,3.009887e-10,5.185712e-14,0.050001,2.122734e-20,1.254288e-09,-1.387347e-11,4.500003e-10,73500.0,10.322676,deep_mq
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,xa.s12.00.mhz.1974-10-14HR00_evid00156,-7.625121e-13,2.674855e-10,5.994249e-09,-6.890003e-09,1.288425e-08,-3.282075e-15,2.674866e-10,4.095598e-14,0.050000,1.463384e-21,1.311422e-09,-7.708328e-12,2.030632e-10,73500.0,10.768398,impact_mq
70,xa.s12.00.mhz.1975-04-12HR00_evid00191,-3.119208e-12,2.257003e-09,6.101882e-08,-6.701207e-08,1.280309e-07,1.155721e-15,2.257005e-09,2.915963e-12,0.050001,-1.390352e-19,1.123896e-08,-1.673393e-11,6.459676e-10,73500.0,10.355789,impact_mq
71,xa.s12.00.mhz.1975-05-04HR00_evid00192,-6.893979e-12,4.731551e-09,1.122859e-07,-1.078822e-07,2.201681e-07,4.451727e-12,4.731556e-09,1.281508e-11,0.050000,2.731781e-21,2.005329e-08,4.863785e-11,5.611433e-10,73500.0,49.535275,impact_mq
72,xa.s12.00.mhz.1975-06-24HR00_evid00196,3.233473e-13,2.872242e-10,1.199012e-08,-1.173117e-08,2.372129e-08,1.064287e-13,2.872244e-10,4.722268e-14,0.050001,-1.540761e-20,1.107912e-09,-5.767059e-12,3.866764e-10,73500.0,10.226565,impact_mq
